## ⚠ TODO: NOT FINISHED

In [7]:
%load_ext autoreload
%autoreload 2
%load_ext autotime
%config InlineBackend.figure_format = 'retina'

import itertools
import os
from collections import defaultdict
from pathlib import Path
from sklearn.model_selection import KFold
from toolz import valmap
from typing import Tuple

for folder in itertools.chain([Path.cwd()], Path.cwd().parents):
    if (folder / 'Pipfile').exists():
        os.chdir(folder)
        break

from notebooks.heart_transplant.dependencies.heart_transplant_data import get_rolling_cv_inputs_cached

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.01 s (started: 2021-03-25 12:38:11 +00:00)


In [8]:
_, y, X, dataset_raw, sampling_sets = get_rolling_cv_inputs_cached()

[Memory]0.9s, 0.0min    : Loading get_rolling_cv_inputs...
time: 1.97 s (started: 2021-03-25 12:38:12 +00:00)


In [ ]:
N_WINDOWS = 15

threshold_years = pipe(
    np.linspace(2010, 2015, N_WINDOWS),
    partial(map, round),
#     append_is_last,
)

for nr, window_year_to in enumerate(threshold_years):

    X_window = X_reduced.assign(tx_year=dataset_raw['tx_year'])


    X_window = X_window[X_window['tx_year'] <= window_year_to]
    X_window = X_window.sort_values(by='tx_year')

    removed_features = []
    for feature in X_window.columns:
        counts = X_window[feature].value_counts()

        if len(counts) < 2:
            removed_features.append(feature)

    print('Removed features:', formatting.format_list(removed_features))
    features = [ column for column in X_window.columns if column not in removed_features ]
    X_window_normalized = X_window[features]



    X_window_level_1 = X_window_normalized[X_window_normalized['tx_year'] <= window_year_to - 5]
    X_window_level_2 = X_window_normalized[
        (X_window_normalized['tx_year'] > window_year_to - 5) & (X_window_normalized['tx_year'] <= window_year_to - 1)
    ]

    X_window_test = X_window_normalized[X_window_normalized['tx_year'] == window_year_to ]

    y_window_level_1 = y_reduced.loc[X_window_level_1.index]
    y_window_level_2 = y_reduced.loc[X_window_level_2.index]
    y_window_test = y_reduced.loc[X_window_test.index]


    print()
    print(window_year_to)
    print(f'Window 1 n={len(X_window_level_1)}')
    print(f'Window 2 n={len(X_window_level_2)}')
    print(f'Window Test n={len(X_window_test)}')
    print()

    lr = make_pipeline(
        get_transform_pipeline(features=features),
        StandardScaler(),
        LogisticRegression(),
    )
    lr.fit(X_window_level_1, y_window_level_1)

    random_forest = get_random_forest_pipeline(features=features)
    random_forest.fit(X_window_level_1, y_window_level_1)

    y_window_level_2_predict_lr = lr.predict_proba(X_window_level_2)[:, 1]
    y_window_level_2_predict_rf = random_forest.predict_proba(X_window_level_2)[:, 1]

    X_predicted_for_level_2 = DataFrame({
        'y_lr': y_window_level_2_predict_lr,
        'y_rf': y_window_level_2_predict_rf,
        'tx_year': X_window_level_2,
    }, index=X_window_level_2.index)

    level2_predictor = make_pipeline(
        get_transform_pipeline(features=features + ['y_lr', 'y_rf'], extra_continuous_features=['y_lr', 'y_rf']),
        StandardScaler(),
        MLPClassifier(),
    )

    level2_predictor.fit(X_predicted_for_level_2, y_window_level_2)

    X_predicted_for_test_lr = lr.predict_proba(X_window_test)[:, 1]
    X_predicted_for_test_rf = random_forest.predict_proba(X_window_test)[:, 1]
    X_predicted_for_test = DataFrame({
        'y_lr': X_predicted_for_test_lr,
        'y_rf': X_predicted_for_test_rf,
        'tx_year': X_window_test['tx_year'],
    })

    y_window_test_predict = level2_predictor.predict_proba(X_predicted_for_test)[:, 1]
    roc_auc = roc_auc_score(y_window_test, y_window_test_predict)
    results_stacked['mlp'][window_year_to] = roc_auc